In [1]:
!pip install bs4
!pip install pdf2image
!pip install ebooklib
!pip install PyPDF2
!pip install NLTK
!pip install pytesseract
!pip install tesseract


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
Using cached pdf2image-1.17.0-py3-none-any.whl (11 kB)

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 1.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for ebooklib: filename=EbookLib-0.18-py3-none-any.whl size=38778 sha256=5fc3020f632c474f26ed91dfac9ef5c4f0d8848ab0bc34b532f31f587681b685
  Stored in directory: /Users/pascalbrissette/Library/Caches/pip/wheels/65/2b/63/68307c736d5a2fafeebe9df3e5eccacfe892204ce1fd31a03c
Successfully built ebooklib

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00a 0:00:01

[notice] A

In [2]:
import os
import time
import re
from bs4 import BeautifulSoup
from pdf2image import convert_from_path
import ebooklib
from ebooklib import epub
from PyPDF2 import PdfReader
from PIL import Image
import pytesseract

In [3]:
def get_file_path():
    """Fonction interactive qui demande le chemin du fichier à importer

    Returns:
        str: nom du chemin vers le fichier
    """
    while True:
        file_path = input("Entre le chemin vers le fichier PDF (.pdf), ePub (.epub) ou text (.txt) : ")
        if file_path.endswith('.pdf') or file_path.endswith('.epub') or file_path.endswith('.txt'):
            if os.path.exists(file_path):
                return file_path
            else:
                print("Le fichier n'existe pas. Entre le bon chemin.")
        else:
            print("Le format indiqué n'est pas adéquat. Fournis un fichier de type .pdf, .epub ou .txt")

In [4]:
def load_txt(file_path):
    """Fonctions de chargement d'un fichier .txt

    Args:
        file_path (str): chemin du fichier

    Returns:
        str: chaine de caractères (texte complet du fichier)
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read().replace('\n', ' ').replace('\xad', '').replace('|', ' ').replace(' – ', ' ').strip()
    return text

def load_pdf(file_path):
    """Océrisation avec PyPDF2

    Args:
        file_path (str): chemin du fichier

    Returns:
        str: chaine de caractères (texte complet du fichier)
    """
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''.join([page.extract_text() for page in reader.pages])
        text = text.replace('-\n', '').replace('\n', ' ').replace('\xad', '').replace('|', ' ').replace(' – ', ' ').strip()
    return text

def extract_text_with_tesseract(file_path, lang='fra'):
    """Océrisation avec tesseract (plus lent qu'avec PyPDF2)

    Args:
        file_path (str): chemin du fichier
        lang (str, optional): Langue du moteur de reconnaissance de caractères. Par défaut 'fra'.

    Raises:
        ValueError: Erreur si fichier le fichier n'est pas traitable par tesseract

    Returns:
        str: chaine de caractères (texte complet du fichier)
    """
    file_extension = file_path.split('.')[-1].lower()
    if file_extension in ['jpg', 'jpeg', 'png', 'bmp', 'tiff']:
        image = Image.open(file_path)
        text = pytesseract.image_to_string(image, lang=lang)
    elif file_extension == 'pdf':
        images = convert_from_path(file_path)
        texts = []
        for image in images:
            page_text = pytesseract.image_to_string(image, lang=lang)
            texts.append(page_text)
        text = '\n'.join(texts)
    else:
        raise ValueError(f"Unsupported file type '{file_extension}'. Supported types are JPG, PNG, BMP, TIFF, and PDF.")
    
    text = re.sub(r'-\s+', '', text)
    
    # Tokenize the text on any kind of whitespace and then join the tokens with a single space
    tokens = text.split()
    text = ' '.join(tokens)
    
    return text


def load_epub(file_path):
    """Extraction du texte de fichiers epub

    Args:
        file_path (str): chemin du fichier

    Returns:
        str: chaine de caractères (texte complet)
    """
    book = epub.read_epub(file_path)
    text = ''
    for item in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
        lines = item.get_content().decode('utf-8').splitlines()
        html_text = ' '.join(lines)
        soup = BeautifulSoup(html_text, 'html.parser')
        text += soup.get_text(separator=' ')
    text = text.replace('-\n', ' ').replace('\n', ' ').replace('\xad', '').replace('|', ' ').replace(' – ', ' ').strip()
    return text

In [5]:
def load_file(file_path):
    """Fonction qui utilise l'une des fonctions de lecture, selon le type de fichier identifié

    Args:
        file_path (str): chemin du fichier

    Raises:
        ValueError: erreur si le fichier n'est pas de type pdf, epub ou txt (exlusion des formats image pour l'instant)

    Returns:
        str: texte
    """
    start_time = time.time()
    file_type = file_path.split('.')[-1]
    if file_type not in ['pdf', 'epub', 'txt']:
        raise ValueError(f"Unsupported file type '{file_type}'")
    
    if file_type == 'txt':
        text = load_txt(file_path)
    elif file_type == 'pdf':
        # text = load_pdf(file_path)
        text = extract_text_with_tesseract(file_path)
    elif file_type == 'epub':
        text = load_epub(file_path)
        
    end_time = time.time()  # End time after loading
    
    duration = end_time - start_time
    print(f"Le fichier a été chargé en {duration:.2f} secondes.")

    return file_type, text


In [6]:
def truncate_text_start(text):
    """Fonction de troncature (début et fin)

    Args:
        text (str): texte

    Returns:
        str: texte tronqué (optionnel)
    """
    while True:
        num_words_to_print = min(1000, len(text.split()) // 2)
        print("DÉBUT DU TEXTE: " + ' '.join(text.split()[:num_words_to_print]))
        truncate_text = input("Veux-tu retrancher une partie du texte? (oui/non): ")
        if truncate_text.lower() == 'oui':
            sequence = input("Indique la dernière séquence de caractères à CONSERVER: ")
            if sequence in text:
                text = text[text.index(sequence):]
                print(f"Le texte a été coupé. La séquence '{sequence}' forme maintenant le début du texte.")
            else:
                print("La séquence de caractères n'a pas été trouvée. Ressaye.")
        elif truncate_text.lower() == 'non' or truncate_text.strip() == '':
            break
        else:
            print("Entrée non valide. Entre 'oui' or 'non'.")
    return text

def truncate_text_end(text):
    while True:
        num_words_to_print = min(1000, len(text.split()) // 2)
        print("FIN DU TEXTE: " + ' '.join(text.split()[-num_words_to_print:]))
        truncate_text = input("Veux-tu couper la fin du texte? (oui/non): ")
        if truncate_text.lower() == 'oui':
            sequence = input("Entre le début de la séquence qui doit être coupée jusqu'à la fin: ")
            if sequence in text:
                text = text[:text.index(sequence)]
                print(f"Le texte a été coupé. La séquence '{sequence}' et tout ce qui vient après a été coupé.")
            else:
                print("La séquence n'a pas été trouvée dans le texte. Essaye à nouveau.")
        elif truncate_text.lower() == 'non' or truncate_text.strip() == '':
            break
        else:
            print("Entrée non valide. Entre 'oui' or 'non'.")
    return text

In [7]:
french_stopwords = {'auront', 'aie', 'd', 'étants', 'ayants', 't', 'étantes', 'et', 'avait', 'j', 'auriez', 'étaient', 'était', 'fusses', 'fussions', 'suis', 'auraient', 'ou', 'seraient', 'été', 'seront', 'te', 'tu', 'pour', 'y', 'êtes', 'avec', 'avons', 'sont', 'sommes', 'avaient', 'je', 'mes', 'eue', 'le', 'de', 'leur', 's', 'soyons', 'aurons', 'même', 'aies', 'elle', 'en', 'fussent', 'dans', 'n', 'eûmes', 'fûtes', 'eut', 'lui', 'm', 'étiez', 'soyez', 'ayant', 'étée', 'eu', 'un', 'ta', 'avez', 'avions', 'aviez', 'sera', 'mais', 'on', 'eusse', 'ayez', 'étées', 'soit', 'aurait', 'eusses', 'ses', 'eues', 'auras', 'ayante', 'aurez', 'serai', 'des', 'aura', 'étante', 'sur', 'ton', 'il', 'avais', 'nos', 'me', 'sa', 'ayons', 'son', 'seriez', 'sois', 'aux', 'serez', 'l', 'aurai', 'es', 'serons', 'eût', 'ces', 'qui', 'aurais', 'eurent', 'étés', 'pas', 'se', 'mon', 'notre', 'étais', 'ont', 'ils', 'fûmes', 'soient', 'eux', 'votre', 'serais', 'fussiez', 'ma', 'eûtes', 'que', 'les', 'ce', 'du', 'à', 'c', 'est', 'seras', 'eussent', 'moi', 'vous', 'ai', 'eussiez', 'la', 'fusse', 'eussions', 'nous', 'vos', 'fut', 'ayantes', 'aient', 'toi', 'qu', 'ne', 'par', 'étions', 'au', 'fût', 'serions', 'eus', 'une', 'serait', 'étant', 'as', 'aurions', 'ait', 'fus', 'tes', 'furent'}
english_stopwords = {'re', 'from', 'and', "won't", 'am', 'her', "hasn't", 'after', 'this', 'shan', 'doesn', 'my', 'himself', 'until', "didn't", 'against', "don't", 'theirs', "you'd", 'is', 'each', 'don', 've', 'into', 'at', 'i', 'or', 'your', 'its', 'needn', 's', 'all', 'again', "shan't", 'mightn', 'being', 'hers', 'his', 'having', 'y', 'she', 'other', 'own', 'haven', "haven't", 'no', 'have', 'not', 'off', 'nor', 'most', 'themselves', 'did', 'with', 'few', 'for', 'once', 'didn', 'd', 'we', 'now', 'which', 'a', 'has', 'him', "hadn't", 'it', 'only', 'how', 'ours', "you've", 'further', 'such', 'as', 'wouldn', 'below', 'isn', 'our', "wasn't", 'yourselves', 'me', "it's", 'both', 'of', 'in', 'ma', "weren't", 'won', 'aren', "you're", 'that', 'between', 'so', 'ain', 'down', 'but', 'their', 'are', 'hasn', "wouldn't", "isn't", 'by', 'too', 'yourself', 'just', 'an', 'should', 'couldn', 'where', 'about', "doesn't", 'm', 'same', 'doing', 'under', 'hadn', 'why', 'during', "needn't", 'if', "you'll", 'than', 'mustn', 'does', 'ourselves', 'shouldn', 'some', 'you', 'been', 'what', 'when', "couldn't", 'will', 'out', "aren't", 'these', 'those', 'itself', "shouldn't", 'while', 'any', 'weren', 'he', 'before', "that'll", 'to', 'over', 'yours', 'they', 'on', 'them', 'herself', 'who', 'the', 'above', 'here', 'o', 'myself', 'up', 't', 'more', "mightn't", "mustn't", 'because', "should've", 'had', 'can', 'whom', 'very', 'through', 'then', 'wasn', "she's", 'there', 'll', 'were', 'was', 'do', 'be'}


def process_text(text):
    """
    Traitement interactif du texte:
    1. Removing punctuation.
    2. Removing numbers.
    3. Removing symbols.
    4. Removing stopwords.
    5. Transforming case.
    6. Converting non-ASCII characters to ASCII.

    Returns:
        str: texte traité (optionnel)
    """
    # 1. Ask user if they want to remove punctuation
    choice = input("Veux-tu supprimer la ponctuation du texte? (oui/non): ").strip().lower()
    if choice == 'oui':
        text = re.sub(r'[.,!?]', ' ', text)
        text = re.sub(r'\s+', ' ', text)

    # 2. Ask user if they want to remove numbers
    choice = input("Veux-tu supprimer les nombres du texte? (oui/non): ").strip().lower()
    if choice == 'oui':
        text = re.sub(r'\b\d+\b', ' ', text)
        text = re.sub(r'\s+', ' ', text)

    # 3. Ask user if they want to remove symbols (excluding punctuation)
    choice = input("Veux-tu supprimer les symboles (hors ponctuation) du texte? (oui/non): ").strip().lower()
    if choice == 'oui':
        text = re.sub(r'[^\w\s.,!?]', ' ', text)
        text = re.sub(r'\s+', ' ', text)

    # 4. Ask user if they want to use lowercase
    choice = input("Veux-tu transformer la casse (minuscules)? (oui/non): ").strip().lower()
    if choice == 'oui':
        text = text.lower()

    # 5. Ask if the user wants to remove stopwords
    remove_stopwords = input("Veux-tu supprimer les mots vides (stopwords) ? (oui/non): ").strip().lower()

    if remove_stopwords == "oui":
        # Ask for the language of the stopwords
        language_choice = input("Quelle langue pour la liste des mots vides ? (fr/an): ").strip().lower()

        if language_choice == "fr":
            current_stopwords = french_stopwords
        elif language_choice == "an":
            current_stopwords = english_stopwords
        else:
            print("Choix de langue non valide. Utilisation des stopwords français par défaut.")
            current_stopwords = french_stopwords

        # Display the chosen stopwords list
        print("\nListe actuelle des mots vides:")
        print(", ".join(sorted(current_stopwords)))

        # Ask if user wants to remove any stopwords from this list iteratively
        while True:
            remove_word = input("Veux-tu enlever des mots de cette liste ? (oui/non): ").strip().lower()

            if remove_word == "oui":
                words_to_remove = input("Entre les mots à enlever, séparés par des virgules: ").strip().lower().split(',')
                for word in words_to_remove:
                    current_stopwords.discard(word.strip())
                print("\nListe mise à jour des mots vides:")
                print(", ".join(sorted(current_stopwords)))
            elif remove_word == "non":
                break

        # Ask if user wants to add new stopwords to the list iteratively
        while True:
            add_word = input("Veux-tu ajouter des mots à cette liste ? (oui/non): ").strip().lower()

            if add_word == "oui":
                words_to_add = input("Entre les mots à ajouter, séparés par des virgules: ").strip().lower().split(',')
                for word in words_to_add:
                    current_stopwords.add(word.strip())
                print("\nListe mise à jour des mots vides:")
                print(", ".join(sorted(current_stopwords)))
            elif add_word == "non":
                break

        # Apply the final stopwords removal
        words = text.split()
        text = ' '.join([word for word in words if word.lower() not in current_stopwords])

    # 6. Ask user if they want to convert text to ASCII
    choice = input("Veux-tu convertir le texte en ASCII (ceci supprimera les caractères spéciaux)? (oui/non): ").strip().lower()
    if choice == 'oui':
        text = text.encode("ascii", errors="ignore").decode()

    return text

In [8]:
def process_text(text):
    """
    Traitement interactif du texte:
    1. Removing punctuation.
    2. Removing numbers.
    3. Removing symbols.
    4. Removing stopwords.
    5. Transforming case.
    6. Converting non-ASCII characters to ASCII.

    Returns:
        str: texte traité (optionnel)
    """

    # 1. Ask user if they want to remove punctuation
    choice = input("Veux-tu supprimer la ponctuation du texte? (oui/non): ").strip().lower()
    if choice == 'oui':
        text = re.sub(r'[.,!?]', ' ', text)
        text = re.sub(r'\s+', ' ', text)

    # 2. Ask user if they want to remove numbers
    choice = input("Veux-tu supprimer les nombres du texte? (oui/non): ").strip().lower()
    if choice == 'oui':
        text = re.sub(r'\b\d+\b', ' ', text)
        text = re.sub(r'\s+', ' ', text)

    # 3. Ask user if they want to remove symbols (excluding punctuation)
    choice = input("Veux-tu supprimer les symboles (hors ponctuation) du texte? (oui/non): ").strip().lower()
    if choice == 'oui':
        text = re.sub(r'[^\w\s.,!?]', ' ', text)
        text = re.sub(r'\s+', ' ', text)

    # 4. Ask user if they want to use lowercase
    choice = input("Veux-tu transformer la casse (minuscules)? (oui/non): ").strip().lower()
    if choice == 'oui':
        text = text.lower()

    # 5. Ask if the user wants to remove stopwords
    remove_stopwords = input("Veux-tu supprimer les mots vides (stopwords) ? (oui/non): ").strip().lower()

    if remove_stopwords == "oui":
        # Ask for the language of the stopwords
        language_choice = input("Quelle langue pour la liste des mots vides ? (fr/an): ").strip().lower()

        if language_choice == "fr":
            current_stopwords = french_stopwords
        elif language_choice == "an":
            current_stopwords = english_stopwords
        else:
            print("Choix de langue non valide. Utilisation des stopwords français par défaut.")
            current_stopwords = french_stopwords

        # Display the chosen stopwords list
        print("\nListe actuelle des mots vides:")
        print(", ".join(sorted(current_stopwords)))

        # Ask if user wants to remove any stopwords from this list iteratively
        while True:
            remove_word = input("Veux-tu enlever des mots de cette liste ? (oui/non): ").strip().lower()

            if remove_word == "oui":
                words_to_remove = input("Entre les mots à enlever, séparés par des virgules: ").strip().lower().split(',')
                for word in words_to_remove:
                    current_stopwords.discard(word.strip())
                print("\nListe mise à jour des mots vides:")
                print(", ".join(sorted(current_stopwords)))
            elif remove_word == "non":
                break

        # Ask if user wants to add new stopwords to the list iteratively
        while True:
            add_word = input("Veux-tu ajouter des mots à cette liste ? (oui/non): ").strip().lower()

            if add_word == "oui":
                words_to_add = input("Entre les mots à ajouter, séparés par des virgules: ").strip().lower().split(',')
                for word in words_to_add:
                    current_stopwords.add(word.strip())
                print("\nListe mise à jour des mots vides:")
                print(", ".join(sorted(current_stopwords)))
            elif add_word == "non":
                break

        # Apply the final stopwords removal
        words = text.split()
        text = ' '.join([word for word in words if word.lower() not in current_stopwords])

    # 6. Ask user if they want to convert text to ASCII
    choice = input("Veux-tu convertir le texte en ASCII (ceci supprimera les caractères spéciaux)? (oui/non): ").strip().lower()
    if choice == 'oui':
        text = text.encode("ascii", errors="ignore").decode()

    return text

In [9]:
def correct_ocr_errors_interactive(text):
    """
    Fonction de correction de mots

    Args:
        text (str): texte

    Returns:
        str: texte corrigé
    """
    continue_outer_loop = True

    while continue_outer_loop:
        # Apply initial corrections to the entire text
        text = re.sub(r"(\w) ’(\w)", r"\1’\2", text)
        text = re.sub(r'’', r"'", text)
        text = re.sub(r' \.', '.', text)
        text = re.sub(r' ,', ',', text)

        # Extract the first 3000 words for inspection and apply the same initial corrections
        words = text.split()
        sample = ' '.join(words[:3000])
        sample = re.sub(r"(\w) ’(\w)", r"\1’\2", sample)
        sample = re.sub(r' \.', '.', sample)
        sample = re.sub(r' ,', ',', sample)

        print("\nFirst 3000 words for inspection:")
        print(sample)

        # Obtenir les patterns à corriger post-océrisation
        common_splits = {}
        while True:
            print("\nVeux-tu corriger une chaîne de caractères problématique ? (oui/non)")
            choice = input().strip().lower()

            if choice == 'oui':
                split_word = input("Entre le mot à corriger (ex.: 'tou jours'): ")
                correct_word = input(f"Comment le mot doit-il être écrit '{split_word}' (ex.: 'toujours'): ")
                common_splits[split_word] = correct_word

                # Apply the correction from the dictionary to the entire text and inform the user about the change
                count = text.count(split_word)
                text = text.replace(split_word, correct_word)
                print(f"Le motif '{split_word}' a été trouvé et corrigé {count} fois en '{correct_word}'.")

            elif choice == 'non':
                continue_outer_loop = False
                break
            else:
                print("Choix non valide. Entre 'oui' ou 'non'.")

        # Print the corrected version of the previously shown sample for verification
        corrected_words = text.split()
        corrected_sample = ' '.join(corrected_words[:3000])

        print("\nVERSION CORRIGÉE DES PREMIERS 3000 MOTS:")
        print(corrected_sample)

    return text

In [10]:
def save_results(text, file_path):
    """Fonction de sauvegarde du texte lu et traité

    Args:
        text (str): chaine de caractères (texte complet)
        file_path (str): chemin du fichier pour la sauvegarde
    """
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)


In [11]:
def process_text_pipeline():
    """
    Fonction maîtresse. Assemblage de toutes les fonctions et exécution.

    """
    # Get file path from user
    file_path = get_file_path()

    # Load the text based on the file type
    _, text = load_file(file_path)

    # Offer user to truncate start and end of the text
    text = truncate_text_start(text)
    text = truncate_text_end(text)

    # Correct errors
    correct_ocr_errors_interactive(text)

    # Process the text
    processed_text = process_text(text)

    words = processed_text.split()
    sample = ' '.join(words[:3000])
    print(sample)

    # Save the results
    output_file_path = input("Entre le chemin pour sauvegarder le résultat: ")
    save_results(processed_text, output_file_path)
    print(f"Le texte a été traité et sauvegardé dans {output_file_path}")

In [12]:
process_text_pipeline()

Le fichier a été chargé en 347.67 secondes.
DÉBUT DU TEXTE: Prologue Montréal, jeudi 3 novembre 1836 En ces premiers jours de novembre, mois des morts, le temps était sombre et maussade, comme l’humeur de l’évêque de Montréal. Depuis le début de 1836, la vie de Mgr Jean-Jacques Lartigue n’était qu’un cauchemar sans fin. Son estomac se serrait souvent et des brûlures lui remontaient constamment dans l’œsophage, le forçant à consommer des quantités astronomiques de lait chaud et de miel. Il dormait peu et mal, comme tous ceux qui n’ont pas la conscience tranquille. Il évitait même les miroirs, qui lui renvoyaient l’image d’un homme aux yeux cernés et au visage creusé par les soucis. En un an, il paraissait avoir vieilli de dix. Tout cela à cause du maudit livre paru en janvier. Le récit scabreux avait suscité une curiosité malsaine non seulement chez les Montréalais, mais à la grandeur de l’Amérique. Des dizaines de milliers d’exemplaires s’étaient envolés en quelques mois, selon ce qu’o